In [1]:
import ijson
import json
import pandas as pd

# Hantera json, eftersom den är stor
""" with open('data/Pub_Lay_Net/labels.json', 'rb') as file:
    sample = file.read(1000).decode('utf-8')
    print(sample) """

df = pd.read_json('data/Pub_Lay_Net/labels.json', orient='records', lines=True)

In [2]:
# tre huvudnycklar
df_images = pd.json_normalize(df['images'].iloc[0])
df_annotations = pd.json_normalize(df['annotations'].iloc[0])
df_categories = pd.json_normalize(df['categories'].iloc[0])

In [3]:
df_merged = df_annotations.merge(df_images, left_on='image_id', right_on='id', suffixes=('_ann', '_img'))
df_full = df_merged.merge(df_categories, left_on='category_id', right_on='id', suffixes=('', '_cat'))

print(f"Full merged dataset shape: {df_full.shape}")
print("Available columns: ", df_full.columns.tolist())
df_full.head()

Full merged dataset shape: (3263046, 14)
Available columns:  ['segmentation', 'area', 'iscrowd', 'image_id', 'bbox', 'category_id', 'id_ann', 'file_name', 'height', 'id_img', 'width', 'supercategory', 'id', 'name']


,segmentation,area,iscrowd,image_id,bbox,category_id,id_ann,file_name,height,id_img,width,supercategory,id,name
0,"[[52.38, 444.87, 291.97, 444.87, 291.97, 456.4...",13787.433083,0,0,"[40.42, 444.87, 251.55, 63.86]",1,0,PMC3866684_00003.jpg,811,0,613,,1,text
1,"[[309.91, 444.91, 561.46, 444.91, 561.46, 456....",3266.252634,0,0,"[309.91, 444.91, 251.55, 21.97]",1,1,PMC3866684_00003.jpg,811,0,613,,1,text
2,"[[52.38, 507.66, 291.97, 507.66, 291.97, 519.1...",8037.095047,0,0,"[40.42, 507.66, 251.55, 32.45]",1,2,PMC3866684_00003.jpg,811,0,613,,1,text
3,"[[321.87, 465.8, 561.46, 465.8, 561.46, 477.34...",18566.000169,0,0,"[309.91, 465.8, 251.55, 74.3]",1,3,PMC3866684_00003.jpg,811,0,613,,1,text
4,"[[40.42, 397.09, 561.44, 397.09, 561.44, 405.5...",17140.380181,0,0,"[40.42, 397.09, 521.02, 34.83]",1,4,PMC3866684_00003.jpg,811,0,613,,1,text


In [5]:
import os

os.makedirs('data/processed/labels/', exist_ok=True)

df_images.to_parquet('data/processed/labels/df_images.parquet')
df_annotations.to_parquet('data/processed/labels/df_annotations.parquet')
df_categories.to_parquet('data/processed/labels/df_categories.parquet')
df_full.to_parquet('data/processed/labels/df_full.parquet')

for file in ['df_images.parquet', 'df_annotations.parquet', 'df_categories.parquet', 'df_full.parquet']:
    size_mb = os.path.getsize(f'data/processed/{file}') / (1024**2)
    print(f"{file}: {size_mb:.1f} MB")

ArrowKeyError: No type extension with name arrow.py_extension_type found

In [ ]:
# För att ladda
""" df_full = pd.read_parquet('data/processed/df_full.parquet')

print(f"Laddade {len(df_full)} annotationer!") """

